# Descriptive analysis of SSNAP Extract Version 2

## Plain English Summary

tbc

## Aims

* Restrict to records from 2017 to 2019 (inclusive), with minimum of 300 admissions and 10 thrombolysis patients per stroke team

## Observations

tbc

## Import libraries

In [16]:
# Import packages and functions
import os
import pandas as pd
from dataclasses import dataclass

# Set the maximum number of columns and rows to 100
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Set paths and filenames

In [17]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and files.'''

    data_path = './../output/'
    data_filename = 'reformatted_data.csv'
    notebook = '01'
    
paths = Paths()

## Load and restrict data

In [18]:
raw_data = pd.read_csv(os.path.join(paths.data_path, paths.data_filename))

Restrict to records from 2017, 2018 and 2019.

In [5]:
print('Number of records per year:')
print(raw_data.year.value_counts().sort_index().to_string())
print('Total: {0}'.format(len(raw_data.index)))

Number of records per year:
2016    56510
2017    58983
2018    58549
2019    60413
2020    59301
2021    66625
Total: 360381


In [6]:
raw_data_restrict = raw_data[raw_data['year'].isin([2017, 2018, 2019])]
print('New total number of records: {0}'.format(len(raw_data_restrict.index)))

New total number of records: 177945


Restrict to stroke teams with a minimum of 300 admissions and 10 thrombolysis patients across that three year period.

In [7]:
keep = []
discard = 0

# Group dataframe by stroke team
groups = raw_data_restrict.groupby('stroke team')

# Loop through name (each stroke team) and group_df (relevant rows from data)
for name, group_df in groups:
    # Skip if admissions less than 300 or thrombolysis patients less than 10
    admissions = len(group_df.index)
    thrombolysis_received = group_df['thrombolysis'] == 1
    if (admissions < 300) or (thrombolysis_received.sum() < 10):
        discard += 1
        continue
    else:
        keep.append(group_df)

# Concatenate output
data = pd.concat(keep)

# Number of stroke teams kept v.s. removed
print('Number of stroke teams remaining in dataset: {0}'.format(len(keep)))
print('Number of stroke teams removed from dataset: {0}'.format(discard))


Number of stroke teams remaining in dataset: 114
Number of stroke teams removed from dataset: 4


## Exploratory (to tidy)

Initially copied from SAMueL 1

In [11]:
print('Total admissions: {0}'.format(len(data.index)))
print('Average yearly admissions: {0}'.format(round(len(data.index)/3)))

Total admissions: 177631
Average yearly admissions: 59210


In [25]:
# Stroke types
data['infarction'].map({1: 'Infarction',
                        0: 'Primary Intracerebral Haemorrage'}).value_counts(normalize=True)

Infarction                          0.874662
Primary Intracerebral Haemorrage    0.125338
Name: infarction, dtype: float64